### **Put this Noteboka and MURA-v1.1 folder within the google drive in My Drive and put this notebook in MURA-v1.1 folder.**


# MURA Challenge

In [0]:
#Load the requird libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plb
import matplotlib.image as mpimg
import datetime as dt
from PIL import Image
from os import listdir
import pandas as pd
import h5py
import os
import csv
import cv2
%matplotlib inline

# **Check if GPU has been allocated and detected **

In [0]:
#import tensorflow as tf
#tf.test.gpu_device_name()

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

# Run the folowing 2 cells to setup Google Drive on Google Colab

In [0]:
#To set up Google Drive on Google Colab
#Will ask for a couple of verification from your gmail to connect to google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
#To set up Google Drive on Google Colab
!mkdir -p drive
!google-drive-ocamlfuse drive

# Loading Path to Images

In [0]:
# Uncomment the following line only if you need to create HDF5 file to store images and their labels
#import Images

# **Loading Images**

In [0]:
current = os.getcwd()
print(current)

In [0]:
os.listdir()

In [0]:
root_Path = os.chdir("drive")
os.listdir()

In [0]:
os.listdir()

In [0]:
os.chdir("MURA-v1.1")

In [0]:
hf = open("Saved_Images.h5")

In [0]:
os.listdir()

In [0]:
hf = h5py.File('Saved_Images.h5', 'r')

In [0]:

x = hf["X1"]
plb.imshow(x)

In [0]:
# Calculate number of images "n"
n = int(len(hf)/2)
n

In [0]:
start = dt.datetime.now()

ig = []
for i in range(0, n):
  img = hf["X" + str(i)]
  ig.append(img)

end = dt.datetime.now()
print("\r", "Loaded Images:", i, "||" ,"Time Elapsed:", (end - start).seconds, "seconds", end = "")

In [0]:
len(ig)